# Лабораторная работа №2 по машинному обучению
### Выполнила Блистунова Валерия
### Группа М8О-401Б-18
#### Постановка задачи
Необходимо реализовать алгоритмы машинного обучения. Применить данные алгоритмы на наборы данных,подготовленных в первой лабораторной работе. Провести анализ полученных моделей, вычислить метрики
классификатора. Произвести тюнинг параметров в случае необходимости. Сравнить полученные результаты с моделями реализованными в scikit-learn. Аналогично построить метрики классификации. Показать, что полученные
модели не переобучились. Также необходимо сделать выводы о применимости данных моделей к вашей задаче.

Подключение необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
import random as rd
import math

from scipy.optimize import fmin_tnc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import classification_report

Подготовка данных (по аналогии с первой лабораторной работой по машинному обучению)

In [2]:
def dataPreparing(data, split):
    X = data[0:1024]['age'].values
    Y = data[0:1024]['thalach'].values
    Z = data[0:1024]['target'].values
    req_data = []
    for i in range(len(X)):
        req_data.append([[X[i], Y[i]],Z[i]])
    train =[]
    test = []
    for i in range (len(req_data)):
        if rd.random() < split:
            train.append(req_data[i])
        else:
            test.append(req_data[i])
    testX = []
    testY = []
    trainX = []
    trainY = []
    for i in range(len(train)):
        trainX.append(train[i][0])
        trainY.append(train[i][1])
    for i in range(len(test)):
        testX.append(test[i][0])
        testY.append(test[i][1])
    trainX = np.array(trainX)
    trainY = np.array(trainY)
    testX = np.array(testX)
    testY = np.array(testY)
    return trainX, trainY, testX, testY
data = pd.read_csv('heart.csv')
trainX, trainY, testX, testY = dataPreparing(data, 0.85)

#### Логистическая регрессия
Логистическая регрессия — это часть машинного обучения, используемая для решения проблемы бинарной классификации. Логистическая регрессионная модель строится на использовании сигмоиды, а не линейном уравнении.

In [3]:
# Сигмоида
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
# Вычисление взвешенной суммы входных данных
def net_input(theta, x):
    return np.dot(x, theta)
# Вычисление вероятности
def probability(theta, x):
    return sigmoid(net_input(theta, x))
# Вычисление функции стоимости для всех обучающих выборок.
def cost_function(theta, x, y):
    m = x.shape[0]
    total_cost = -(1 / m) * np.sum(y * np.log(probability(theta, x)) + (1 - y) * np.log(1 - probability(theta, x)))
    return total_cost
# Вычисление градиента
def gradient(theta, x, y):
    m = x.shape[0]
    return (1 / m) * np.dot(x.T, sigmoid(net_input(theta, x)) - y)
# Функция, которая будет использоваться для поиска параметров модели, которая минимизирует функцию стоимости
def fit(x, y, theta):
    opt_weights = fmin_tnc(func=cost_function, x0=theta, fprime=gradient,args=(x, y), messages=0)
    return opt_weights[0]
# Точность модели
def prediction(x, param):
    theta = param[:, np.newaxis]
    return probability(theta, x)
def predict(res, probab_threshold=0.5):
    predicted_classes = []
    for pred in res:
        if pred >= probab_threshold:
            predicted_classes += [1]
        else:
            predicted_classes += [0]
    return predicted_classes
def accur(res, trueX):
    count = 0
    for i in range(len(trueX)):
        if ((res[i] >=0.5 and trueX[i] == 1) or (res[i] < 0.5 and trueX[i] == 0)):
            count+=1
    return count / len(trueX)

In [4]:
# Библиотечная функция
model = LogisticRegression(solver='liblinear', random_state=0).fit(trainX, trainY)
pred = model.predict(testX)
test_report = classification_report(pred, testY)
print('sklearn log: \n', pred)
print('accuracy: ', model.score(testX,testY))
print(test_report)
# Результаты собственной реализации
train_log = np.c_[np.ones(len(trainX)), trainX]
test_log = np.c_[np.ones(len(testX)), testX]
theta = np.zeros((train_log.shape[1], 1))
param = fit(train_log, trainY, theta)
res = prediction(test_log, param)
pred_class = predict(res)
test_report = classification_report(pred_class, testY)
print('log:\n', pred_class)
print('accuracy: ', accur(res, testY))
print(test_report)

sklearn log: 
 [1 0 0 1 1 1 1 1 0 1 1 1 0 1 1 0 0 0 1 0 1 0 1 0 1 0 0 0 1 0 1 1 1 1 1 0 0
 1 1 1 1 0 1 1 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 1 0 1 1 1 1 0 0 0 1 0 1 1 0 1
 0 1 1 0 0 1 1 0 1 1 1 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 1 0 0 0 1 1 1 1 0 1
 0 0 0 1 0 0 0 0 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 0 1 1 1 1 0 1 0 0 0]
accuracy:  0.6458333333333334
              precision    recall  f1-score   support

           0       0.64      0.65      0.64        71
           1       0.65      0.64      0.65        73

    accuracy                           0.65       144
   macro avg       0.65      0.65      0.65       144
weighted avg       0.65      0.65      0.65       144

log:
 [1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0

#### Дерево решений
Деревья решений — это модели машинного обучения, используемые для прогнозирования через циклический просмотр каждой функции в наборе данных один за одним.
Построение дерева
1. Выбор атрибута, по которому будет производиться разбиение в данном узле
2. Выбор критерия остановки обучения
3. Выбор метода отсечения ветвей
4. Оценка точности построенного дерева

In [5]:
# Класс узла
class Node:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None
# Дерево решений
class DecisionTreeClassifier:
    def __init__(self, max_depth=1000):
        self.max_depth = max_depth
    def fit(self, X, y, rand=0):
        self.n_classes_ = len(set(y))
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y, rand)
    def predict(self, X):
        return [self._predict(inputs) for inputs in X]
    def _best_split(self, X, y):
        m = y.size
        if m <= 1:
            return None, None
        num_parent = [np.sum(y == c) for c in range(self.n_classes_)]
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None
        for idx in range(self.n_features_):
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes_
            num_right = num_parent.copy()
            for i in range(1, m):
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum((num_left[x] / i) ** 2 for x in range(self.n_classes_))
                gini_right = 1.0 - sum((num_right[x] / (m - i)) ** 2 for x in range(self.n_classes_))
                gini = (i * gini_left + (m - i) * gini_right) / m
                if thresholds[i] == thresholds[i - 1]:
                    continue
                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        return best_idx, best_thr
    def random_best_split(self, X, y):
        m = y.size
        if m <= 1:
            return None, None
        num_parent = [np.sum(y == c) for c in range(self.n_classes_)]
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None
        l = list(range(self.n_features_))
        random.shuffle(l)
        l = np.array(l)
        feat_num = int(np.ceil(np.sqrt(self.n_features_)))
        l = np.resize(l, feat_num)
        for idx in l:
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes_
            num_right = num_parent.copy()
            for i in range(1, m):
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum((num_left[x] / i) ** 2 for x in range(self.n_classes_))
                gini_right = 1.0 - sum((num_right[x] / (m - i)) ** 2 for x in range(self.n_classes_))
                gini = (i * gini_left + (m - i) * gini_right) / m
                if thresholds[i] == thresholds[i - 1]:
                    continue
                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        return best_idx, best_thr
    def _grow_tree(self, X, y, depth=0, rand=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(predicted_class=predicted_class)
        if depth < self.max_depth:
            if rand:
                idx, thr = self.random_best_split(X, y)    
            else:
                idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node
    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

In [6]:
# Библиотечная функция
model = tree.DecisionTreeClassifier()
model.fit(trainX,trainY)
pred = model.predict(testX)
print('sklearn Tree: \n')
print(pred)
print('accuracy: ', model.score(testX,testY))
test_report = classification_report(pred, testY)
print(test_report)
# Собственная реализация
model = DecisionTreeClassifier(max_depth=5)
model.fit(trainX, trainY)
pred = model.predict(testX)
print('Tree: \n')
print(pred)
print('accuracy: ', accur(pred, testY))
test_report = classification_report(pred, testY)
print(test_report)

sklearn Tree: 

[0 1 0 0 0 1 0 1 1 0 1 1 0 1 1 1 0 0 1 1 1 0 1 0 0 0 1 0 0 0 1 0 1 1 1 0 0
 1 1 1 1 0 1 1 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 1 0 0 0 1 0 0 0 1 1 1 1 0 0
 0 1 1 1 1 0 1 0 1 1 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 1 0 1
 0 0 1 0 1 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 0]
accuracy:  0.9583333333333334
              precision    recall  f1-score   support

           0       0.94      0.97      0.96        70
           1       0.97      0.95      0.96        74

    accuracy                           0.96       144
   macro avg       0.96      0.96      0.96       144
weighted avg       0.96      0.96      0.96       144

Tree: 

[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0

#### Random Forest
Случайный лес — набор деревьев решения, которые используют случайные порядки объектов в наборе данных. Используя случайные наборы характеристик для каждого дерева в случайном лесу, мы декоррелируем деревья и дисперсия полученной модели уменьшается, что, несомненно, является плюсом данного алгоритма.

In [7]:
class RandomTreeClassifier:
    def __init__(self, tree_num=10):
        self.tree_num = tree_num
        self.trees = []
    def get_sub_x(self, X,y):
        x_size = len(X)
        sample = []
        new_y = []
        while len(sample) < x_size:
            ind = rd.randrange(x_size)
            sample.append(X[ind])
            new_y.append(y[ind])
        sample = np.array(sample)
        new_y = np.array(new_y)
        return sample, new_y
    def fit(self, X, y):
        for i in range(self.tree_num):
            tree = DecisionTreeClassifier()
            X1, Y1 = self.get_sub_x(X, y)
            tree.fit(X1,Y1, 1)
            self.trees.append(tree)
    def predict(self, X):
        res = np.zeros(len(X))
        for i in range(self.tree_num):
            res += self.trees[i].predict(X)

        for i in range(len(res)):
            if (res[i] / self.tree_num < 0.5):
                res[i] = 0
            else:
                res[i] = 1
        return res

In [8]:
# Библиотечная функция
model = RandomForestClassifier(n_estimators=10)
model.fit(trainX, trainY)
pred = model.predict(testX)
print('sklearn Random Forest: \n')
print(pred)
print('accuracy: ', model.score(testX,testY))
test_report = classification_report(pred, testY)
print(test_report)
# Результат собственной реализации
model = RandomTreeClassifier(5)
model.fit(trainX, trainY)
pred = model.predict(testX)
print('Random Forest: \n')
print(pred.astype(int))
print('accuracy: ', accur(pred, testY))
test_report = classification_report(pred, testY)
print(test_report)

sklearn Random Forest: 

[0 1 0 0 0 1 0 1 1 0 1 1 0 1 1 1 0 0 1 1 1 0 1 0 0 0 1 0 0 0 1 0 1 1 1 0 0
 1 1 1 1 0 1 1 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 1 0 0 0 1 0 0 0 1 1 1 1 0 0
 0 1 1 1 1 0 1 0 1 1 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 1 0 1
 0 0 1 0 1 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 0]
accuracy:  0.9583333333333334
              precision    recall  f1-score   support

           0       0.94      0.97      0.96        70
           1       0.97      0.95      0.96        74

    accuracy                           0.96       144
   macro avg       0.96      0.96      0.96       144
weighted avg       0.96      0.96      0.96       144

Random Forest: 

[0 1 0 0 0 1 0 1 1 0 1 1 0 1 1 1 0 0 1 1 1 0 1 0 0 0 1 0 0 0 1 0 1 1 1 0 0
 1 1 1 1 0 1 1 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 1 0 0 0 1 0 0 0 1 1 1 1 0 0
 0 1 1 1 1 0 1 0 1 1 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 1 0 1
 0 0 1 0 1 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 0]
accuracy:  0.95

#### Вывод
Результаты собственных реализаций не сильно отличаются от функций в библиотеке sklearn. Каждый из алгоритмов имеет свои преимущества и недостатки, поэтому следует выбирать алгоритм под определенную задачу, набор данных.

#### Список литературы
1. https://www.machinelearningmastery.ru/building-a-logistic-regression-in-python-301d27367c24/
2. https://habr.com/ru/post/509472/
3. https://www.machinelearningmastery.ru/implement-decision-tree-algorithm-scratch-python/